In [1]:
import os
import pandas as pd
import numpy as np
from pulp import *

In [2]:
# n = number of branches
# m = #types of trucks
# Nt[t] = number of trucks of type t
# K = total number of trucks (including all types) 
# Kt[k] gives the type of kth truck
# cost[i] = cost matrix for truck[i]
# demand_w[i] = weight demand of ith branch
# demand_v[i] = volume demand of ith branch 

In [3]:
demandFile = 'DP2/demand.xlsx'
truckFile = 'DP2/TruckCap.xlsx'
costFile = 'DP2/cost.xlsx'
   
Truck = pd.read_excel(truckFile)
trucks = Truck.columns[1:]
m = trucks.shape[0]
t_w = Truck.values[2][1:]
t_v = Truck.values[1][1:]

# branches 
branches = pd.read_excel(costFile, trucks[0]).columns
n = len(branches)

# cost[i] = cost matrix for truck[i]
cost = []
for t in trucks:
    Cost = pd.read_excel(costFile, t).values
    cost.append(Cost)

    
Demand = pd.read_excel(demandFile)
br = [i.split(' ')[0] for i in Demand.index[1:]]
Demand = Demand.values
d_w = {}
d_v = {}
for i in range(len(br)):
    d_w[br[i]] = Demand[i+1][0]
    d_v[br[i]] = Demand[i+1][1]

# demand_w[i] = weight demand, demand_v[i] = volume demand of ith branch 
demand_w = [0]
demand_v = [0]

for i in branches[1:]:
    
    demand_w.append(d_w[i])
    demand_v.append(d_v[i])


In [4]:
def make_prob(n, K):
    prob = LpProblem('VRP',LpMinimize)
    #    x_k_i_j    where, k for #truck,    i,j for index (i,j)
    x = []
    for k in range(K):
        xi = []
        for i in range(n):
            xij = []
            for j in range(n):
                a = 'x_'+str(k)+'_'+str(i)+'_'+str(j)
                a = LpVariable(a, lowBound = 0, upBound = 1, cat = 'Integer')
                xij.append(a)
            xi.append(xij)
        x.append(xi)
    
    return prob, x

In [5]:
def get_values(y, n, K):
    x = []
    for k in range(K):
        xi = []
        for i in range(n):
            xij = []
            for j in range(n):
                a = value(y[k][i][j])
                xij.append(a)
            xi.append(xij)
        x.append(xi)
    return x

In [6]:
Nt = [n for t in trucks];        # Nt[k] = number of trucks of type k
K = sum(Nt)
Kt = []                          # Kt[i] gives type of kth truck
for i in range(m):
    for j in range(Nt[i]):
        Kt.append(i)
mat = [(i,j) for i in range(n) for j in range(n)]

prob, x = make_prob(n,K)

# Objective function
s = 0
for k in range(K):
    s += sum([cost[Kt[k]][i][j]*x[k][i][j] for i,j in mat])

prob+= s

#Constraint 1
for i in range(1,n):
    s = sum([x[k][i][j] for k in range(K) for j in range(n)])
    prob+= s==1


#Constraint 2
for j in range(1,n):
    s = sum([x[k][i][j] for k in range(K) for i in range(n)])
    prob+= s==1


#Constraint 3
for k in range(K):
    s = sum([x[k][0][j] for j in range(n)])
    prob+= s==1

#Constraint 4
for k in range(K):
    s = sum([x[k][i][0] for i in range(n)])
    prob+= s==1

#Constraint 5 
for k in range(K):
    s_w=0
    s_v=0
    for i in range(n):
        s_w = s_w + sum([x[k][i][j]*demand_w[j] for j in range(n)])
        s_v = s_v + sum([x[k][i][j]*demand_v[j] for j in range(n)])
    prob+= s_w <= t_w[Kt[k]]
    prob+= s_v <= t_v[Kt[k]]
    
#Constraint 6
for k in range(K):
    s_w_l=0
    s_v_l=0
    s_w_r=0
    s_v_r=0
    for i in range(n):
        s_w_l = s_w_l + sum([x[k][i][j]*demand_w[j] for j in range(n)])
        s_v_l = s_v_l + sum([x[k][i][j]*demand_v[j] for j in range(n)])
    for j in range(n):
        s_w_r = s_w_r + sum([x[k][i][j]*demand_w[i] for i in range(n)])
        s_v_r = s_v_r + sum([x[k][i][j]*demand_v[i] for i in range(n)])
    prob+= s_w_l + (-1*s_w_r) == 0
    prob+= s_v_l + (-1*s_v_r) == 0    

#Constraint 7
for k in range(K):
    for i in range(1, n):
        for j in range(i,n):
            prob+= x[k][i][j]+x[k][j][i]<=1
            
#Constraint 8
for k in range(K):
    s = sum([x[Kt[k]][i][j] for i in range(1,n) for j in range(1,n)])
    prob+= s<=2
            
    
prob

VRP:
MINIMIZE
5100*x_0_0_1 + 6900*x_0_0_10 + 3180*x_0_0_11 + 1650*x_0_0_12 + 6930*x_0_0_13 + 8280*x_0_0_14 + 1980*x_0_0_15 + 735*x_0_0_2 + 3120*x_0_0_3 + 7290*x_0_0_4 + 4380*x_0_0_5 + 6090*x_0_0_6 + 4350*x_0_0_7 + 5220*x_0_0_8 + 1350*x_0_0_9 + 6900*x_0_10_0 + 1650*x_0_10_1 + 3720*x_0_10_11 + 6000*x_0_10_12 + 4650*x_0_10_13 + 3630*x_0_10_14 + 4950*x_0_10_15 + 6660*x_0_10_2 + 4920*x_0_10_3 + 6810*x_0_10_4 + 3450*x_0_10_5 + 3180*x_0_10_6 + 5340*x_0_10_7 + 3330*x_0_10_8 + 6690*x_0_10_9 + 3180*x_0_11_0 + 2070*x_0_11_1 + 3720*x_0_11_10 + 2310*x_0_11_12 + 4080*x_0_11_13 + 6450*x_0_11_14 + 1260*x_0_11_15 + 2970*x_0_11_2 + 2340*x_0_11_3 + 5160*x_0_11_4 + 2280*x_0_11_5 + 4470*x_0_11_6 + 2280*x_0_11_7 + 2400*x_0_11_8 + 2760*x_0_11_9 + 1650*x_0_12_0 + 4350*x_0_12_1 + 6000*x_0_12_10 + 2310*x_0_12_11 + 5280*x_0_12_13 + 8490*x_0_12_14 + 1770*x_0_12_15 + 1170*x_0_12_2 + 3780*x_0_12_3 + 5700*x_0_12_4 + 5700*x_0_12_5 + 6300*x_0_12_6 + 2790*x_0_12_7 + 3690*x_0_12_8 + 2490*x_0_12_9 + 6930*x_0_13_0 + 4290*

In [ ]:
status = prob.solve()
LpStatus[status]
x = get_values(x, n, K)

In [ ]:
X = []
for i in x:
    for j in i:
        X.append(j)
    X.append([])
df=pd.DataFrame(X)
df.to_excel('output.xlsx')